In [1]:
# pip install smart_open boto3 pillow

In [23]:
import boto3
from smart_open import open
import zipfile
import io
import os
import glob

# Set up the S3 client
s3 = boto3.client('s3')

# Define the S3 bucket and key
bucket_name = 'ieee-dataport'
object_key = 'data/7540/StairNet.zip'

# Define the directory to save images
local_image_dir = os.path.expanduser('~/SageMaker/data/images')

# Function to check if data already exists
def data_exists(local_dir):
    # Check for the presence of any image files in the directory
    images = glob.glob(os.path.join(local_dir, '*'))
    if images:
        return True
    return False

# Function to stream and extract images
def stream_and_extract_images(bucket, key, local_dir):
    buffer_size = 10 * 1024 * 1024  # 10 MB buffer size for reading chunks

    # Use smart_open to stream the zip file
    with open(f's3://{bucket}/{key}', 'rb') as s3_file:
        # BufferedReader with a buffer size
        with zipfile.ZipFile(io.BufferedReader(s3_file, buffer_size=buffer_size)) as z:
            # Iterate over the files in the zip
            for file_info in z.infolist():
                if file_info.filename.endswith(('.png', '.jpg', '.jpeg')):
                    try:
                        with z.open(file_info) as image_file:
                            img_save_path = os.path.join(local_dir, os.path.basename(file_info.filename))
                            # Save only if the file doesn't exist
                            if not os.path.exists(img_save_path):
                                print(f"Extracting {file_info.filename}...")
                                with open(img_save_path, 'wb') as f:
                                    f.write(image_file.read())
                    except Exception as e:
                        print(f"Failed to extract {file_info.filename}: {e}")

# Create the local directory if it does not exist
os.makedirs(local_image_dir, exist_ok=True)

# Check if the data already exists
if data_exists(local_image_dir):
    print(f"Data already exists in {local_image_dir}. Skipping download and extraction.")
else:
    print(f"Data does not exist in {local_image_dir}. Downloading and extracting...")
    # Stream and extract images
    stream_and_extract_images(bucket_name, object_key, local_image_dir)

print(f"Data check complete. Images are available in {local_image_dir}")


Data already exists in /home/ec2-user/SageMaker/data/images. Skipping download and extraction.
Data check complete. Images are available in /home/ec2-user/SageMaker/data/images


In [24]:
import os

# Define the directory containing the images
image_directory = '/home/ec2-user/SageMaker/data/images'

# Initialize counters for each suffix
lg_count = 0
lg_is_count = 0
is_count = 0
is_lg_count = 0

# List all files in the directory
for filename in os.listdir(image_directory):
    # Check and count for each suffix
    if filename.endswith(' LG.jpg'):
        lg_count += 1
    elif filename.endswith(' LGIS.jpg'):
        lg_is_count += 1
    elif filename.endswith(' IS.jpg'):
        is_count += 1
    elif filename.endswith(' ISLG.jpg'):
        is_lg_count += 1

# Print the counts
print(f"Number of images ending with 'LG': {lg_count}")
print(f"Number of images ending with 'LG-IS': {lg_is_count}")
print(f"Number of images ending with 'IS': {is_count}")
print(f"Number of images ending with 'IS-LG': {is_lg_count}")


Number of images ending with 'LG': 442360
Number of images ending with 'LG-IS': 15888
Number of images ending with 'IS': 48179
Number of images ending with 'IS-LG': 9025


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.manual_seed(42)
np.random.seed(42)

# Rest of your code will go here

Using device: cpu


## Step 2:

- First, let's define the dataset structure 

dataset_structure = {
    'IS': 48179,
    'ISLG': 9025,
    'LG': 442360,
    'LGIS': 15888
}

total_samples = sum(dataset_structure.values())
print(f"Total samples: {total_samples}")

- Now, let's create functions to split the data according to the given percentages:

In [26]:
# def split_dataset(dataset_structure, train_pct=0.895, val_pct=0.035, test_pct=0.07):
#     train_set, val_set, test_set = {}, {}, {}
    
#     for class_name, total_samples in dataset_structure.items():
#         train_samples = int(total_samples * train_pct)
#         val_samples = int(total_samples * val_pct)
#         test_samples = total_samples - train_samples - val_samples
        
#         train_set[class_name] = train_samples
#         val_set[class_name] = val_samples
#         test_set[class_name] = test_samples
    
#     return train_set, val_set, test_set

# train_set, val_set, test_set = split_dataset(dataset_structure)

# print("Training set:", train_set)
# print("Validation set:", val_set)
# print("Test set:", test_set)

In [27]:
import random

def split_data(file_list, train_pct=0.895, val_pct=0.035, test_pct=0.07):
    random.shuffle(file_list)
    total = len(file_list)
    train_end = int(total * train_pct)
    val_end = train_end + int(total * val_pct)
    
    return file_list[:train_end], file_list[train_end:val_end], file_list[val_end:]

- For loading the data, we'll need to create a custom dataset class

In [28]:
import os
from PIL import Image
from torch.utils.data import Dataset

class StairNetDataset(Dataset):
    def __init__(self, root_dir, file_list, transform=None):
        self.root_dir = root_dir
        self.file_list = file_list
        self.transform = transform
        self.class_to_idx = {'IS': 0, 'ISLG': 1, 'LG': 2, 'LGIS': 3}

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_name = self.file_list[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        # Extract label from filename
        parts = img_name.split()
        label = parts[-1].split('.')[0]  # Get the last part before the file extension
        
        # Handle the case where the label might be two words (e.g., 'LG IS')
        if len(parts) > 2 and parts[-2] in ['LG', 'IS']:
            label = f"{parts[-2]} {label}"
        
        label_idx = self.class_to_idx[label]

        if self.transform:
            image = self.transform(image)

        return image, label_idx

- creat a stratified dataset
- setup the dataloaders

In [29]:
import random
from collections import defaultdict
from torchvision import transforms
from torch.utils.data import DataLoader

def split_data_stratified(file_list, train_pct=0.895, val_pct=0.035, test_pct=0.07):
    # Group files by class
    class_files = defaultdict(list)
    for file in file_list:
        parts = file.split()
        label = parts[-1].split('.')[0]
        if len(parts) > 2 and parts[-2] in ['LG', 'IS']:
            label = f"{parts[-2]} {label}"
        class_files[label].append(file)
    
    train_files, val_files, test_files = [], [], []
    
    for class_label, files in class_files.items():
        random.shuffle(files)
        n_files = len(files)
        n_train = int(n_files * train_pct)
        n_val = int(n_files * val_pct)
        
        train_files.extend(files[:n_train])
        val_files.extend(files[n_train:n_train+n_val])
        test_files.extend(files[n_train+n_val:])
    
    random.shuffle(train_files)
    random.shuffle(val_files)
    random.shuffle(test_files)
    
    return train_files, val_files, test_files

# Define transforms
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Set up data
root_dir = '/home/ec2-user/SageMaker/data/images'  
all_files = os.listdir(root_dir)
image_files = [f for f in all_files if f.endswith('.jpg')]

# Split data
train_files, val_files, test_files = split_data_stratified(image_files)

# Create datasets
train_dataset = StairNetDataset(root_dir, train_files, transform=train_transform)
val_dataset = StairNetDataset(root_dir, val_files, transform=val_transform)
test_dataset = StairNetDataset(root_dir, test_files, transform=val_transform)

# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")

Number of training samples: 461328
Number of validation samples: 18039
Number of test samples: 36085


- check the distribution (caution: takes very very long time to count! so run this if you have a lot of time availale)

In [ ]:
## Print class distribution in each split

# def print_class_distribution(dataset):
#     class_counts = defaultdict(int)
#     for _, label in dataset:
#         class_counts[label] += 1
#     total = sum(class_counts.values())
#     for label, count in class_counts.items():
#         print(f"Class {label}: {count} ({count/total:.2%})")

# print("\nTraining set class distribution:")
# print_class_distribution(train_dataset)
# print("\nValidation set class distribution:")
# print_class_distribution(val_dataset)
# print("\nTest set class distribution:")
# print_class_distribution(test_dataset)